<a href="https://colab.research.google.com/github/Raswanth-Prasath/NGSIM-Driving-Behavior-Analysis/blob/main/NGSIM_Driving_Behavior_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import List, Optional, Dict, Tuple

@dataclass
class CarFollowingPair:
    """
    Represents a validated car-following pair with all relevant information
    """
    leader_id: int
    follower_id: int
    
    start_frame: int
    end_frame: int
    lane_id: int
    
    metrics: Dict = None

    @property
    def duration(self) -> float:
        """Duration of car-following episode in seconds"""
        return (self.end_frame - self.start_frame + 1) * 0.1  # Convert frames to seconds

class CarFollowingIdentifier:
    """
    Identifies and validates car-following pairs in trajectory data
    """
    def __init__(self, min_duration: float = 20.0,
                 min_spacing: float = 2.0,
                 max_spacing: float = 100.0):
        """
        Initialize with validation criteria
        
        Parameters:
        min_duration: Minimum duration in seconds for valid car-following
        min_spacing: Minimum allowed spacing between vehicles (meters)
        max_spacing: Maximum allowed spacing between vehicles (meters)
        """
        self.min_duration = min_duration
        self.min_frames = int(min_duration * 10)  # Convert to frames (0.1s intervals)
        self.min_spacing = min_spacing
        self.max_spacing = max_spacing
        
    def identify_pairs(self, df: pd.DataFrame) -> List[CarFollowingPair]:
        """
        Main method to identify valid car-following pairs, with integrated leader-follower validation
        
        Parameters:
        df: DataFrame with columns for Vehicle_ID, Frame_ID, Lane_ID, LocalY, Leader_ID, Follower_ID
        
        Returns:
        List of validated CarFollowingPair objects
        """
        valid_pairs = []
        pairs_before_validation = 0
        pairs_after_validation = 0
        
        # Step 1: Group data by lane
        for lane_id, lane_data in df.groupby('Lane_ID'):
            # Skip special lanes (like merge lanes or shoulders)
            if lane_id > 6:  # Assuming regular lanes are 1-6
                continue
                
            # Step 2: Process each time window in the lane
            frames = sorted(lane_data['Frame_ID'].unique())
            
            # Step 3: Track ongoing pairs
            current_pairs = {}
            
            for frame in frames:
                frame_data = lane_data[lane_data['Frame_ID'] == frame]
                
                # Sort vehicles by position to identify leader-follower relationships
                frame_vehicles = frame_data.sort_values('LocalY', ascending=False)
                
                # Step 4: Check each consecutive pair of vehicles
                for i in range(len(frame_vehicles) - 1):
                    leader = frame_vehicles.iloc[i]
                    follower = frame_vehicles.iloc[i + 1]
                    
                    # Calculate spacing
                    spacing = leader['LocalY'] - follower['LocalY']
                    pairs_before_validation += 1
                    
                    # Get vehicle IDs for relationship validation
                    leader_id = leader['Vehicle_ID']
                    follower_id = follower['Vehicle_ID']
                    follower_leader_id = follower['Leader_ID']
                    leader_follower_id = leader['Follower_ID']
                    
                    # Validate the leader-follower relationship:
                    # 1. If explicit relationships exist (non -1), they must be correct
                    # 2. If no explicit relationships (-1), allow validation by spacing
                    valid_relationship = False
                    
                    if follower_leader_id != -1 or leader_follower_id != -1:
                        # At least one explicit relationship exists - it must be correct
                        leader_match = follower_leader_id == leader_id
                        follower_match = leader_follower_id == follower_id
                        valid_relationship = leader_match or follower_match
                    else:
                        # No explicit relationships - allow validation by spacing
                        valid_relationship = True
                    
                    # Both the relationship and spacing must be valid
                    if valid_relationship and self.min_spacing <= spacing <= self.max_spacing:
                        pairs_after_validation += 1
                        pair_id = (leader_id, follower_id)
                        
                        if pair_id not in current_pairs:
                            # Start new pair tracking
                            current_pairs[pair_id] = {
                                'start_frame': frame,
                                'current_frame': frame,
                                'lane_id': lane_id
                            }
                        else:
                            # Update existing pair
                            current_pairs[pair_id]['current_frame'] = frame
                    else:
                        # Invalid pair - end pair if exists
                        pair_id = (leader_id, follower_id)
                        self._check_and_add_pair(current_pairs, pair_id, valid_pairs)
                
            # Process any remaining pairs at the end of the lane
            for pair_id in list(current_pairs.keys()):
                self._check_and_add_pair(current_pairs, pair_id, valid_pairs)
        
        print(f"Pairs before validation: {pairs_before_validation}")
        print(f"Pairs after validation: {pairs_after_validation}")
        return valid_pairs
    
    def _check_and_add_pair(self, current_pairs: Dict, 
                           pair_id: Tuple[int, int],
                           valid_pairs: List[CarFollowingPair]) -> None:
        """
        Validates and adds a car-following pair if it meets duration criteria
        """
        if pair_id in current_pairs:
            pair_data = current_pairs[pair_id]
            duration_frames = pair_data['current_frame'] - pair_data['start_frame'] + 1
            
            if duration_frames >= self.min_frames:
                # Create validated pair
                valid_pairs.append(CarFollowingPair(
                    leader_id=pair_id[0],
                    follower_id=pair_id[1],
                    start_frame=pair_data['start_frame'],
                    end_frame=pair_data['current_frame'],
                    lane_id=pair_data['lane_id']
                ))
            
            # Remove pair from tracking
            del current_pairs[pair_id]
    
    def compute_pair_metrics(self, pair: CarFollowingPair, df: pd.DataFrame) -> Dict:
        """
        Computes detailed metrics for a validated car-following pair with proper handling of zero speeds
        
        Parameters:
        pair: CarFollowingPair object
        df: Original trajectory DataFrame
        
        Returns:
        Dictionary of computed metrics
        """
        # Get leader and follower trajectories
        leader_data = df[(df['Vehicle_ID'] == pair.leader_id) & 
                        (df['Frame_ID'] >= pair.start_frame) & 
                        (df['Frame_ID'] <= pair.end_frame)]
        
        follower_data = df[(df['Vehicle_ID'] == pair.follower_id) & 
                        (df['Frame_ID'] >= pair.start_frame) & 
                        (df['Frame_ID'] <= pair.end_frame)]
        
        # Compute spacing statistics
        spacing = leader_data['LocalY'].values - follower_data['LocalY'].values
        
        # Compute speed difference statistics
        speed_diff = leader_data['Speed'].values - follower_data['Speed'].values
        
        # Compute time headway with careful handling of zero speeds
        follower_speeds = follower_data['Speed'].values
        
        # Create a mask for non-zero speeds to avoid division by zero
        non_zero_speed_mask = follower_speeds > 0.01  # Small threshold to handle near-zero speeds
        
        # Only calculate time headway where speed is non-zero
        if np.any(non_zero_speed_mask):
            # Calculate time headway only for non-zero speeds
            valid_spacings = spacing[non_zero_speed_mask]
            valid_speeds = follower_speeds[non_zero_speed_mask]
            time_headway = valid_spacings / valid_speeds
            
            # Remove any extreme values that might still occur
            reasonable_headway_mask = (time_headway > 0) & (time_headway < 20)  # Filter unreasonable values
            valid_headway = time_headway[reasonable_headway_mask] if np.any(reasonable_headway_mask) else np.array([])
        else:
            # No valid speeds for calculation
            valid_headway = np.array([])
        
        # Handle empty arrays for edge cases
        if len(valid_headway) == 0:
            headway_stats = {
                'mean': np.nan,
                'std': np.nan,
                'min': np.nan,
                'max': np.nan
            }
        else:
            headway_stats = {
                'mean': np.mean(valid_headway),
                'std': np.std(valid_headway),
                'min': np.min(valid_headway),
                'max': np.max(valid_headway)
            }
        
        return {
            'spacing': {
                'mean': np.mean(spacing),
                'std': np.std(spacing),
                'min': np.min(spacing),
                'max': np.max(spacing)
            },
            'speed_difference': {
                'mean': np.mean(speed_diff),
                'std': np.std(speed_diff),
                'min': np.min(speed_diff),
                'max': np.max(speed_diff)
            },
            'time_headway': headway_stats
        }

def main():
    # Example usage
    # Read the data file
    df = pd.read_csv("D:\ASU Academics\Traffic Flow Theroy\MP-1\Reconstructed NGSIM I80-1 data\Data\DATA (NO MOTORCYCLES).txt", delimiter='\s+', header=None,
                     names=['Vehicle_ID', 'Frame_ID', 'Lane_ID', 'LocalY',
                           'Speed', 'Acceleration', 'Vehicle_Length',
                           'Vehicle_Class', 'Follower_ID', 'Leader_ID'])
    
    # Initialize identifier
    identifier = CarFollowingIdentifier(
        min_duration=20.0,  # 20 seconds minimum
        min_spacing=2.0,    # 2 meters minimum spacing
        max_spacing=100.0   # 100 meters maximum spacing
    )
    
    # Find car-following pairs
    all_pairs = identifier.identify_pairs(df)
    
    # Print summary
    print(f"Found {len(all_pairs)} valid car-following pairs")
    
    # Export all pairs to CSV
    export_pairs_to_csv(all_pairs, df, identifier, "all_car_following_pairs.csv")
    
    # Select the specific pairs we want
    selected_pairs = []
    selected_pair_ids = [
        (1087.0, 1101.0),  # Pair 57 - Aggressive Following with Large Speed Difference
        (120.0, 125.0),    # Pair 13 - Very Close Following
        (2066.0, 2074.0),  # Pair 96 - Conservative Following with Large Spacing
        (1463.0, 1478.0),  # Pair 714 - Lane 3 with Medium Following Distance
        (260.0, 267.0)     # Pair 238 - Stable Following with Small Speed Difference
    ]
    
    # Find these pairs in our identified pairs
    for pair in all_pairs:
        if (pair.leader_id, pair.follower_id) in selected_pair_ids:
            selected_pairs.append(pair)
    
    print(f"Selected {len(selected_pairs)} specific car-following pairs for analysis")
    
    # Export selected pairs to a separate CSV
    export_pairs_to_csv(selected_pairs, df, identifier, "selected_car_following_pairs.csv")
    
    # Analyze each selected pair
    for i, pair in enumerate(selected_pairs):
        print(f"\nPair {i+1}:")
        print(f"Leader ID: {pair.leader_id}")
        print(f"Follower ID: {pair.follower_id}")
        print(f"Duration: {pair.duration:.1f} seconds")
        print(f"Lane: {pair.lane_id}")
        
        # Compute and print metrics
        metrics = identifier.compute_pair_metrics(pair, df)
        print("\nMetrics:")
        for metric, values in metrics.items():
            print(f"\n{metric}:")
            for stat, value in values.items():
                print(f"  {stat}: {value:.2f}")

def export_pairs_to_csv(pairs, df, identifier, filename):
    """
    Export all car-following pairs with their metrics to a CSV file
    
    Parameters:
    pairs: List of CarFollowingPair objects
    df: Original trajectory DataFrame
    identifier: CarFollowingIdentifier instance for computing metrics
    filename: Name of the output CSV file
    """
    # Prepare a list to hold all pair data
    all_pairs_data = []
    
    # Process each pair
    for i, pair in enumerate(pairs):
        # Compute metrics for this pair
        metrics = identifier.compute_pair_metrics(pair, df)
        
        # Create a dictionary with all relevant data
        pair_data = {
            'pair_id': i + 1,
            'leader_id': pair.leader_id,
            'follower_id': pair.follower_id,
            'start_frame': pair.start_frame,
            'end_frame': pair.end_frame,
            'lane_id': pair.lane_id,
            'duration_seconds': pair.duration,
            'mean_spacing': metrics['spacing']['mean'],
            'std_spacing': metrics['spacing']['std'],
            'min_spacing': metrics['spacing']['min'],
            'max_spacing': metrics['spacing']['max'],
            'mean_speed_diff': metrics['speed_difference']['mean'],
            'std_speed_diff': metrics['speed_difference']['std'],
            'min_speed_diff': metrics['speed_difference']['min'],
            'max_speed_diff': metrics['speed_difference']['max'],
            'mean_time_headway': metrics['time_headway']['mean'],
            'std_time_headway': metrics['time_headway']['std'],
            'min_time_headway': metrics['time_headway']['min'],
            'max_time_headway': metrics['time_headway']['max']
        }
        
        all_pairs_data.append(pair_data)
    
    # Convert to DataFrame
    pairs_df = pd.DataFrame(all_pairs_data)
    
    # Save to CSV
    pairs_df.to_csv(filename, index=False)
    print(f"Exported {len(pairs)} car-following pairs to {filename}")
        
if __name__ == "__main__":
    main()

Pairs before validation: 966462
Pairs after validation: 963929
Found 2029 valid car-following pairs
Exported 2029 car-following pairs to all_car_following_pairs.csv
Selected 5 specific car-following pairs for analysis
Exported 5 car-following pairs to selected_car_following_pairs.csv

Pair 1:
Leader ID: 120.0
Follower ID: 125.0
Duration: 28.1 seconds
Lane: 1

Metrics:

spacing:
  mean: 17.80
  std: 6.47
  min: 10.29
  max: 28.33

speed_difference:
  mean: 0.28
  std: 1.40
  min: -3.13
  max: 3.17

time_headway:
  mean: 1.30
  std: 0.23
  min: 1.00
  max: 1.78

Pair 2:
Leader ID: 1087.0
Follower ID: 1101.0
Duration: 20.1 seconds
Lane: 1

Metrics:

spacing:
  mean: 37.24
  std: 17.02
  min: 17.05
  max: 65.33

speed_difference:
  mean: 2.25
  std: 1.86
  min: -2.29
  max: 5.32

time_headway:
  mean: 2.14
  std: 0.68
  min: 1.25
  max: 3.24

Pair 3:
Leader ID: 2066.0
Follower ID: 2074.0
Duration: 22.3 seconds
Lane: 1

Metrics:

spacing:
  mean: 45.91
  std: 8.10
  min: 35.46
  max: 64.52


Analyzes statistical properties of car-following pairs and generates comprehensive visualizations and analyses.

**Car-Following Visualization and Calibration**

The following code uses the car_following_pairs.csv file generated from the previous code to analyze the car-following pairs and plot the results.

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import logging
import os

# Configure logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

def load_data(file_path):
    """Load and process the trajectory data"""
    try:
        # Read data with numbered columns first
        data = pd.read_csv(file_path, delimiter='\s+', header=None)
        
        # Rename columns to match expected format
        data.columns = ['Vehicle_ID', 'Frame_ID', 'Lane_ID', 'Local_Y', 
                       'Speed', 'Acceleration', 'Vehicle_Length',
                       'Vehicle_Class', 'Follower_ID', 'Leader_ID']
        
        logger.info(f"Successfully loaded data with {len(data)} rows")
        return data
        
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        return pd.DataFrame()

def load_selected_pairs(csv_path):
    """Load previously identified car-following pairs from CSV"""
    try:
        pairs_df = pd.read_csv(csv_path)
        
        # Convert DataFrame rows to list of pair dictionaries
        selected_pairs = []
        for _, row in pairs_df.iterrows():
            pair = {
                'leader_id': row['leader_id'],
                'follower_id': row['follower_id'],
                'start_frame': row['start_frame'],
                'end_frame': row['end_frame'],
                'lane': row['lane_id'],
                'duration': row['duration_seconds'],
                'description': f"Mean spacing: {row['mean_spacing']:.1f}m, Mean time headway: {row['mean_time_headway']:.1f}s"
            }
            selected_pairs.append(pair)
            
        logger.info(f"Loaded {len(selected_pairs)} car-following pairs from {csv_path}")
        return selected_pairs
        
    except Exception as e:
        logger.error(f"Error loading selected pairs: {e}")
        return []


def process_pair_data(data, selected_pairs):
    """Process data for selected car-following pairs"""
    pair_data = []
    
    for i, pair in enumerate(selected_pairs):
        # Get trajectory data for leader and follower
        frames = range(int(pair['start_frame']), int(pair['end_frame']) + 1)
        
        leader_data = data[
            (data['Vehicle_ID'] == pair['leader_id']) & 
            (data['Frame_ID'].isin(frames))
        ].sort_values('Frame_ID')
        
        follower_data = data[
            (data['Vehicle_ID'] == pair['follower_id']) & 
            (data['Frame_ID'].isin(frames))
        ].sort_values('Frame_ID')
        
        # Combine data
        merged_data = pd.merge(
            leader_data, 
            follower_data,
            on='Frame_ID',
            suffixes=('_leader', '_follower')
        )
        
        # Skip pairs with no merged data
        if len(merged_data) == 0:
            logger.warning(f"Skipping pair {i+1} ({pair['leader_id']}-{pair['follower_id']}) due to empty merged data")
            continue
            
        time_data = []
        for _, row in merged_data.iterrows():
            time_data.append({
                'time': (row['Frame_ID'] - pair['start_frame']) * 0.1,
                'leader_speed': row['Speed_leader'],
                'follower_speed': row['Speed_follower'],
                'leader_accel': row['Acceleration_leader'],
                'follower_accel': row['Acceleration_follower'],
                'spacing': row['Local_Y_leader'] - row['Local_Y_follower'] - row['Vehicle_Length_leader'],
                'relative_speed': row['Speed_leader'] - row['Speed_follower']
            })
        
        pair_data.append({
            'pair_id': f"{pair['leader_id']}-{pair['follower_id']}",
            'description': pair.get('description', ''),
            'lane': pair['lane'],
            'duration': pair['duration'],
            'time_data': pd.DataFrame(time_data)
        })
    
    return pair_data

def plot_pair_visualizations(pair_data, output_dir="."):
    """Create professional visualizations for car-following pairs suitable for reports"""
    
    # Set global matplotlib style for report-quality figures
    plt.style.use('seaborn-v0_8-whitegrid')
    
    # Define consistent colors
    LEADER_COLOR = '#0B2447'  # Dark blue
    FOLLOWER_COLOR = '#FF6B6B'  # Coral red
    SPACING_COLOR = '#2E8B57'  # Sea green
    RELATIVE_COLOR = '#800080'  # Purple
    
    for pair in pair_data:
        time_data = pair['time_data']
        
        # Create figure with adjusted size and spacing
        fig = plt.figure(figsize=(12, 16))
        gs = fig.add_gridspec(4, 1, figure=fig, height_ratios=[1, 1, 1, 1], hspace=0.4)
        
        gs.update(top=0.90, bottom=0.15, hspace=0.3)
        
        # Add main title with padding
        title = f"Car-Following Pair Analysis\n Leader {pair['pair_id'].split('-')[0]} - Follower {pair['pair_id'].split('-')[1]}"
        # if pair['description']:
        #     title += f"\n{pair['description']}"
        fig.suptitle(title, fontsize=14, y=0.95, fontweight='bold')
        
        # Speed profiles
        ax1 = fig.add_subplot(gs[0])
        ax1.plot(time_data['time'], time_data['leader_speed'], 
                label='Leader', color=LEADER_COLOR, linewidth=2)
        ax1.plot(time_data['time'], time_data['follower_speed'], 
                label='Follower', color=FOLLOWER_COLOR, linestyle='--', linewidth=2)
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel('Speed (m/s)')
        ax1.set_title('Speed Profiles', pad=10, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.legend(bbox_to_anchor=(1., 1), loc='upper left', framealpha=0.9, borderaxespad=0.)
        
        # Acceleration profiles
        ax2 = fig.add_subplot(gs[1])
        ax2.plot(time_data['time'], time_data['leader_accel'], 
                label='Leader', color=LEADER_COLOR, linewidth=2)
        ax2.plot(time_data['time'], time_data['follower_accel'], 
                label='Follower', color=FOLLOWER_COLOR, linestyle='--', linewidth=2)
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('Acceleration (m/s²)')
        ax2.set_title('Acceleration Profiles', pad=10, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.legend(bbox_to_anchor=(1., 1), loc='upper left', framealpha=0.9, borderaxespad=0.)
        
        # Space gap
        ax3 = fig.add_subplot(gs[2])
        ax3.plot(time_data['time'], time_data['spacing'], 
                color=SPACING_COLOR, linewidth=2)
        ax3.set_xlabel('Time (s)')
        ax3.set_ylabel('Space Gap (m)')
        ax3.set_title('Following Distance', pad=10, fontweight='bold')
        ax3.grid(True, alpha=0.3)
        
        # Relative speed
        ax4 = fig.add_subplot(gs[3])
        ax4.plot(time_data['time'], time_data['relative_speed'], 
                color=RELATIVE_COLOR, linewidth=2)
        ax4.set_xlabel('Time (s)')
        ax4.set_ylabel('Relative Speed (m/s)')
        ax4.set_title('Relative Speed (Leader - Follower)', pad=10, fontweight='bold')
        ax4.grid(True, alpha=0.3)
        
        # Create statistics text box
        stats_text = (
            f"Analysis Summary:\n"
            f"Lane: {int(pair['lane'])}  |  Duration: {pair['duration']:.1f} seconds\n\n"
            f"Space Gap Statistics:\n"
            f"• Mean: {time_data['spacing'].mean():.2f} m\n"
            f"• Std Dev: {time_data['spacing'].std():.2f} m\n\n"
            f"Speed Dynamics:\n"
            f"• Mean Relative Speed: {time_data['relative_speed'].mean():.2f} m/s\n"
            f"• Mean Follower Accel: {time_data['follower_accel'].mean():.2f} m/s²\n"
            f"• Accel Range: [{time_data['follower_accel'].min():.2f}, "
            f"{time_data['follower_accel'].max():.2f}] m/s²"
        )
        
        # Adjust layout to prevent overlapping
        plt.tight_layout(rect=[0, 0.08, 1, 0.93])
        
        # Save high-resolution figure
        output_path = f"{output_dir}/pair_{pair['pair_id']}_analysis.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        logger.info(f"Saved visualization to {output_path}")
        plt.close()

# Define optimization bounds for each pair
bounds_120_125 = [
    (-0.01, 0.01),  # α: Narrowed to center of error valley for better convergence
    (1.5, 2.5),      # τ: Reduced upper bound as higher values showed diminishing returns
    (0, 0.80)      # β: Tightened around observed optimal performance region
]
bounds_1087_1101 = [
    (-0.05, 0.09),  # α: Focused on region with lowest combined error
    (1.5, 2.5),      # τ: Adjusted to better match observed car-following behavior
    (-0.2, 0.4)     # β: Narrowed to capture optimal stability characteristics
]
bounds_2066_2074 = [
    (-0.01, 0.04),  # α: Concentrated around minimum error point in contours
    (1.5, 2.5),      # τ: Reduced range to more practical following times
    (0.05, 0.32)     # β: Tightened around region with best combined performance
]
bounds_260_267 = [
    (-0.01, 0.1),  # α: Centered on clearest error minimum from contours
    (1.5, 2.5),      # τ: Adjusted for more realistic following behavior
    (-0.05, 0.05)        # β: Focused on region with balanced stability and performance
]
bounds_1463_1478 = [
    (0.9, 1.1),  # α: Narrowed to region with most consistent performance
    (1.5, 2.5),      # τ: Aligned with practical following time observations
    (-0.675, -0.475)     # β: Concentrated on stable region with good error metrics
]
default_bounds = [
    (0.05, 0.5),  # α: Wide range to allow exploration
    (1.0, 3.0),  # τ: Practical range for following time
    (0.5, 1.5)  # β: Reasonable range for relative speed sensitivity
]


pair_bounds = {
    "120.0-125.0": bounds_120_125,
    "1087.0-1101.0": bounds_1087_1101,
    "2066.0-2074.0": bounds_2066_2074,
    "260.0-267.0": bounds_260_267,
    "1463.0-1478.0": bounds_1463_1478,
    "default_bounds": default_bounds
}

def calibrate_cth_rv_model(pair_data, output_dir="."):
    """
    Calibrate the Constant-Time-Headway Relative-Velocity (CTH-RV) model and create
    professional visualizations of the results.
    """
    from scipy.optimize import minimize
    import numpy as np
    
    # Set visualization style for professional output
    plt.style.use('seaborn-v0_8-whitegrid')
    
    # Define professional color scheme
    COLORS = {
        'leader': '#0B2447',      # Dark blue
        'follower': '#FF6B6B',    # Coral red
        'simulated': '#2E8B57',   # Sea green
        'grid': '#E5E5E5'         # Light gray
    }
    calibration_results = {}
    
    for pair in pair_data:
        time_data = pair['time_data']
        pair_id = pair['pair_id']
        
        # Use pair-specific bounds if available, otherwise use default bounds
        bounds = pair_bounds.get(pair_id, default_bounds)
        
        # Extract data for calibration
        times = time_data['time'].values
        spacings = time_data['spacing'].values
        follower_speeds = time_data['follower_speed'].values
        follower_accels = time_data['follower_accel'].values
        relative_speeds = time_data['relative_speed'].values
        
        # Initial parameter guess
        initial_params = [0.1, 1.5, 0.5]  # [α, τ, β]
        
        # Objective function to minimize (RMSE)
        def objective(params):
            alpha, tau, beta = params
            
            # Simulate follower trajectory with current parameters
            simulated_speeds = [follower_speeds[0]]
            simulated_accels = []
            simulated_spacings = [spacings[0]]
            dt = 0.1  # 0.1 seconds between frames
            
            for i in range(1, len(times)):
                # Current state
                v_prev = simulated_speeds[-1]
                s_prev = simulated_spacings[-1]
                
                # CTH-RV model
                accel = alpha * (s_prev - tau * v_prev) + beta * relative_speeds[i-1]
                simulated_accels.append(accel)
                
                # Update speed and position (Euler integration)
                v_new = v_prev + accel * dt
                s_new = s_prev + relative_speeds[i-1] * dt
                
                simulated_speeds.append(v_new)
                simulated_spacings.append(s_new)
            
            # Calculate RMSE for speed, spacing, and acceleration
            speed_error = np.sqrt(np.mean((np.array(simulated_speeds) - follower_speeds)**2))
            spacing_error = np.sqrt(np.mean((np.array(simulated_spacings) - spacings)**2))
            
            # Ensure arrays have the same length for acceleration comparison
            if len(simulated_accels) > 0:
                min_length = min(len(simulated_accels), len(follower_accels[1:]))
                accel_error = np.sqrt(np.mean(
                    (np.array(simulated_accels[:min_length]) - follower_accels[1:min_length+1])**2
                ))
            else:
                accel_error = 0
            
            # Combined error (weighted sum)
            combined_error = 0.4 * speed_error + 0.4 * spacing_error + 0.2 * accel_error
            return combined_error
        
        # Perform optimization
        result = minimize(objective, initial_params, bounds=bounds, method='L-BFGS-B')
        
        if result.success:
            # Extract calibrated parameters
            alpha, tau, beta = result.x
            
            # Store results
            calibration_results[pair_id] = {
                'alpha': alpha,
                'tau': tau,
                'beta': beta,
                'rmse': result.fun
            }
            
            # Generate simulated trajectory with calibrated parameters
            simulated_speeds = [follower_speeds[0]]
            simulated_accels = []
            simulated_spacings = [spacings[0]]
            dt = 0.1  # 0.1 seconds between frames
            
            for i in range(1, len(times)):
                # Current state
                v_prev = simulated_speeds[-1]
                s_prev = simulated_spacings[-1]
                
                # CTH-RV model
                accel = alpha * (s_prev - tau * v_prev) + beta * relative_speeds[i-1]
                simulated_accels.append(accel)
                
                # Update speed and position
                v_new = v_prev + accel * dt
                s_new = s_prev + relative_speeds[i-1] * dt
                
                simulated_speeds.append(v_new)
                simulated_spacings.append(s_new)
            
            # Ensure lengths match for plotting
            simulated_accels.append(simulated_accels[-1] if simulated_accels else 0)
            
            # Calculate specific errors
            speed_rmse = np.sqrt(np.mean((np.array(simulated_speeds) - follower_speeds)**2))
            spacing_rmse = np.sqrt(np.mean((np.array(simulated_spacings) - spacings)**2))
            
            # Match array lengths for acceleration comparison
            if len(simulated_accels) > 0:
                min_length = min(len(simulated_accels), len(follower_accels[1:]))
                accel_rmse = np.sqrt(np.mean(
                    (np.array(simulated_accels[:min_length]) - follower_accels[1:min_length+1])**2
                ))
            else:
                accel_rmse = 0
                
            # Calculate stability criterion
            stability_criterion = (alpha * tau)**2 + 2*alpha*tau*beta - 2*alpha
            is_stable = stability_criterion >= 0
            stability_status = "STABLE" if is_stable else "UNSTABLE"
            
            # Store additional metrics
            calibration_results[pair_id].update({
                'speed_rmse': speed_rmse,
                'spacing_rmse': spacing_rmse,
                'accel_rmse': accel_rmse,
                'string_stable': is_stable,
                'stability_criterion': stability_criterion,
                'stability_status': stability_status
            })
            
            # First, create figure with proper spacing for text box
            fig = plt.figure(figsize=(16, 20))
            # Create a gridspec that leaves room for the text box
            gs = plt.GridSpec(3, 1, figure=fig, height_ratios=[1, 1, 1])
            gs.update(top=0.90, bottom=0.15, hspace=0.3)
            
            # Add descriptive title
            title = (f"CTH-RV Model Calibration Results\n"
                    f"Leader {pair['pair_id'].split('-')[0]} - Follower {pair['pair_id'].split('-')[1]}")
            # if pair.get('description'):
            #     title += f"\n{pair['description']}"
            fig.suptitle(title, fontsize=14, y=0.95, fontweight='bold')
            
            # Create the subplots using the gridspec
            ax1 = fig.add_subplot(gs[0])
            ax2 = fig.add_subplot(gs[1])
            ax3 = fig.add_subplot(gs[2])
            
            # Speed profiles with enhanced styling
            ax1 = fig.add_subplot(gs[0])
            ax1.plot(times, time_data['leader_speed'], 
                    label='Leader (Actual)', color=COLORS['leader'], linewidth=2)
            ax1.plot(times, follower_speeds,
                    label='Follower (Actual)', color=COLORS['follower'], linewidth=2)
            ax1.plot(times, simulated_speeds,
                    label='Follower (Simulated)', color=COLORS['simulated'],
                    linestyle='--', linewidth=2)
            ax1.set_xlabel('Time (s)')
            ax1.set_ylabel('Speed (m/s)')
            ax1.set_title('Speed Profiles', pad=10, fontweight='bold')
            ax1.grid(True, color=COLORS['grid'])
            ax1.legend(bbox_to_anchor=(1, 1),loc='upper left', framealpha=0.9)
            
            # Space gap comparison
            ax2 = fig.add_subplot(gs[1])
            ax2.plot(times, spacings,
                    label='Actual', color=COLORS['leader'], linewidth=2)
            ax2.plot(times, simulated_spacings,
                    label='Simulated', color=COLORS['simulated'],
                    linestyle='--', linewidth=2)
            ax2.set_xlabel('Time (s)')
            ax2.set_ylabel('Space Gap (m)')
            ax2.set_title('Following Distance Comparison', pad=10, fontweight='bold')
            ax2.grid(True, color=COLORS['grid'])
            ax2.legend(bbox_to_anchor=(1, 1), loc='upper left', framealpha=0.9)
            
            # Acceleration comparison
            ax3 = fig.add_subplot(gs[2])
            ax3.plot(times[1:], follower_accels[1:],
                    label='Actual', color=COLORS['leader'], linewidth=2)
            ax3.plot(times[1:], simulated_accels[:-1],
                    label='Simulated', color=COLORS['simulated'],
                    linestyle='--', linewidth=2)
            ax3.set_xlabel('Time (s)')
            ax3.set_ylabel('Acceleration (m/s²)')
            ax3.set_title('Acceleration Profile Comparison', pad=10, fontweight='bold')
            ax3.grid(True, color=COLORS['grid'])
            ax3.legend(bbox_to_anchor=(1, 1), loc='upper left', framealpha=0.9)
            
            # Adjust layout to prevent overlapping
            plt.tight_layout(rect=[0, 0.08, 1, 0.93])
                        
            # Save high-resolution calibration results
            output_path = f"{output_dir}/pair_{pair_id}_calibration.png"
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            
            # Create separate parameter card with enhanced design
            fig_card = plt.figure(figsize=(8, 6))
            ax_card = fig_card.add_subplot(111)
            ax_card.axis('off')
            
            # Add decorative background
            rect = patches.FancyBboxPatch(
                (0.05, 0.05), 0.9, 0.9,
                boxstyle=patches.BoxStyle("Round", pad=0.02),
                facecolor='#F8F9FA',
                edgecolor='#DEE2E6',
                alpha=0.9,
                transform=fig_card.transFigure
            )
            fig_card.patches.append(rect)
            
            # Create comprehensive parameter card text
            card_text = (
                f"Car-Following Model Analysis Summary\n"
                f"{'='*40}\n\n"
                f"Pair Information:\n"
                f"• Leader ID: {pair_id.split('-')[0]}\n"
                f"• Follower ID: {pair_id.split('-')[1]}\n"
                f"• Lane: {int(pair['lane'])}\n"
                f"• Duration: {pair['duration']:.1f} seconds\n\n"
                f"CTH-RV Model Parameters:\n"
                f"• α (sensitivity): {alpha:.4f}\n"
                f"• τ (time headway): {tau:.4f} s\n"
                f"• β (relative velocity): {beta:.4f}\n\n"
                f"Stability Analysis:\n"
                f"• Status: {stability_status}\n"
                f"• Criterion (α²τ² + 2ατβ - 2α): {stability_criterion:.4f}\n\n"
                f"Model Performance:\n"
                f"• Spacing RMSE: {spacing_rmse:.3f} m\n"
                f"• Speed RMSE: {speed_rmse:.3f} m/s\n"
                f"• Acceleration RMSE: {accel_rmse:.3f} m/s²"
            )
            
            plt.figtext(0.5, 0.5, card_text,
                       ha='center', va='center',
                       fontsize=11,
                       fontfamily='monospace',
                       linespacing=1.5)
            
            # Save parameter card
            output_card_path = f"{output_dir}/pair_{pair_id}_parameters.png"
            plt.savefig(output_card_path, dpi=300, bbox_inches='tight')
            
            logger.info(f"Saved calibration results to {output_path}")
            logger.info(f"Saved parameter card to {output_card_path}")
            plt.close('all')
                
        else:
            logger.warning(f"Calibration failed for pair {pair_id}: {result.message}")
            
        
        if result.success:
            # After regular calibration
            pareto_results = perform_pareto_analysis(pair, output_dir, tau)  # Note: passing single pair
            
            if pareto_results and pair_id in pareto_results:
                p_results = pareto_results[pair_id]
                logger.info(f"\nPareto analysis results for pair {pair_id}:")
                logger.info(f"Alpha: {p_results['optimal_alpha']:.4f}")
                logger.info(f"Beta: {p_results['optimal_beta']:.4f}")
                logger.info(f"Stability boundary: β > {p_results['stability_boundary']:.4f}")
                   
        
    return calibration_results

def check_string_stability(alpha, tau, beta):
    """
    Check if a car-following model with given parameters is string stable
    using the complete criterion from Monteil et al. 2018
    
    The string stability criterion is:
    α²τ² + 2ατβ - 2α ≥ 0
    
    Returns True if string stable, False otherwise
    """
    criterion = (alpha * tau)**2 + 2*alpha*tau*beta - 2*alpha
    return criterion >= 0

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


def perform_pareto_analysis(pair_data, output_dir=".", tau=1.5):
    """
    Perform Pareto analysis for CTH-RV model parameters with pair-specific ranges.
    
    Args:
        pair_data: List of dictionaries containing trajectory data for each pair
        output_dir: Directory to save output visualizations
    """
    if not isinstance(pair_data, list):
        pair_data = [pair_data]
    
    all_results = {}
        
    for pair in pair_data:
        try:
            time_data = pair['time_data']
            pair_id = pair['pair_id']
            
            
            # Define pair-specific parameter ranges
            parameter_ranges = {
                "120.0-125.0": {
                    "alpha": (-0.01, 0.01),
                    "beta": (0, 0.80) 
                },
                "1087.0-1101.0": {
                    "alpha": (-0.05, 0.09),
                    "beta": (-0.2, 0.4)
                },
                "2066.0-2074.0": {
                    "alpha": (-0.01, 0.04),
                    "beta": (0.05, 0.32) 
                },
                "260.0-267.0": {
                    "alpha": (-0.01, 0.1),
                    "beta": (-0.05, 0.05) 
                },
                "1463.0-1478.0": {
                    "alpha":  (0.9, 1.1),
                    "beta": (-0.675, -0.475) 
                }
            }
            
            # Get pair-specific parameter ranges
            if pair_id in parameter_ranges:
                alpha_min, alpha_max = parameter_ranges[pair_id]["alpha"]
                beta_min, beta_max = parameter_ranges[pair_id]["beta"]
            else:
                # Use default ranges if pair not found
                alpha_min, alpha_max = 0.125, 0.200
                beta_min, beta_max = 0.95, 1.10
            
            # Create parameter grids with pair-specific ranges
            alpha_range = np.linspace(alpha_min, alpha_max, 20)
            beta_range = np.linspace(beta_min, beta_max, 20)     
            alpha_grid, beta_grid = np.meshgrid(alpha_range, beta_range)
            
            # Convert values to numpy arrays if they aren't already
            times = np.array(time_data['time'])
            actual_speeds = np.array(time_data['follower_speed'])
            actual_spacings = np.array(time_data['spacing'])
            relative_speeds = np.array(time_data['relative_speed'])
            
            # Initialize error arrays
            speed_errors = np.zeros_like(alpha_grid)
            spacing_errors = np.zeros_like(alpha_grid)
            combined_errors = np.zeros_like(alpha_grid)
            
            # Simulate trajectories for each parameter combination
            for i, alpha in enumerate(alpha_range):
                for j, beta in enumerate(beta_range):
                    # Initialize simulation arrays
                    simulated_speeds = [actual_speeds[0]]
                    simulated_spacings = [actual_spacings[0]]
                    dt = 0.1  # timestep
                    
                    # Simulate vehicle trajectory
                    for t in range(1, len(times)):
                        # Current state
                        v_prev = simulated_speeds[-1]
                        s_prev = simulated_spacings[-1]
                        
                        # CTH-RV model
                        accel = alpha * (s_prev - tau * v_prev) + beta * relative_speeds[t-1]
                        
                        # Update states
                        v_new = v_prev + accel * dt
                        s_new = s_prev + relative_speeds[t-1] * dt
                        
                        simulated_speeds.append(v_new)
                        simulated_spacings.append(s_new)
                    
                    # Calculate errors
                    speed_errors[j,i] = np.sqrt(np.mean(
                        (np.array(simulated_speeds) - actual_speeds)**2
                    ))
                    spacing_errors[j,i] = np.sqrt(np.mean(
                        (np.array(simulated_spacings) - actual_spacings)**2
                    ))
                    combined_errors[j,i] = 0.5 * speed_errors[j,i] + 0.5 * spacing_errors[j,i]
            
            # Find optimal parameters
            min_idx = np.unravel_index(np.argmin(combined_errors), combined_errors.shape)
            optimal_alpha = alpha_range[min_idx[1]]  # Note: using min_idx[1] for alpha index
            optimal_beta = beta_range[min_idx[0]]    # Note: using min_idx[0] for beta index
            
            # Create visualizations
            create_pareto_visualizations(
                alpha_grid, beta_grid,
                speed_errors, spacing_errors, combined_errors,
                alpha, beta, tau, pair_id, output_dir
            )
            
            # Store results for this pair
            all_results[pair_id] = {
                'optimal_alpha': optimal_alpha,
                'optimal_beta': optimal_beta,
                'min_combined_error': combined_errors[min_idx],
                'stability_boundary': ((alpha * tau)**2 + 2*alpha*tau*beta - 2*alpha)
            }
            
            logger.info(f"Completed Pareto analysis for pair {pair_id}")
            
        except KeyError as e:
            logger.error(f"Missing required data field for pair {pair_id}: {e}")
            continue
        except Exception as e:
            logger.error(f"Error performing Pareto analysis for pair {pair_id}: {e}")
            continue
    
    # Return results for all pairs
    return all_results


def create_pareto_visualizations(alpha_grid, beta_grid, speed_errors, 
                               spacing_errors, combined_errors, alpha, beta, tau, pair_id, output_dir):
    """
    Create and save Pareto analysis visualizations.
    """
    
    # Create contour plots
    fig, ax1 = plt.subplots(1, 1, figsize=(8, 6))
    
    # Speed error contours
    c1 = ax1.contour(alpha_grid, beta_grid, speed_errors, levels=30)
    ax1.clabel(c1, inline=True, fontsize=8)
    ax1.set_xlabel('α (alpha)')
    ax1.set_ylabel('β (beta)')
    ax1.set_title('Speed Error Contours')
    
    # Add stability boundary to all plots
    min_idx = np.unravel_index(np.argmin(combined_errors), combined_errors.shape)
    optimal_alpha = alpha_grid[min_idx]
    optimal_beta = beta_grid[min_idx]
    
    for ax in [ax1]:
        # ax.axhline(y=beta_boundary, color='r', linestyle='--', label='Stability Boundary')
        ax.plot(optimal_alpha, optimal_beta, 'r*', label='Optimal Point', markersize=10)
        ax.legend()
    
    plt.suptitle(f'Parameter Contour Analysis for CTH-RV Model\nPair {pair_id}',
                 fontsize=14, y=1.05)
    plt.tight_layout()
    
    # Save contour plots
    output_contour_path = f"{output_dir}/pair_{pair_id}_pareto_contours.png"
    plt.savefig(output_contour_path, dpi=300, bbox_inches='tight')
    plt.close()

def main():
    # Define file paths
    data_file = "D:/ASU Academics/Traffic Flow Theroy/MP-1/Reconstructed NGSIM I80-1 data/Data/DATA (NO MOTORCYCLES).txt"
    pairs_file = "selected_car_following_pairs.csv"
    output_dir = "visualizations"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Load trajectory data
    logger.info("Loading trajectory data...")
    data = load_data(data_file)
    
    if len(data) == 0:
        logger.error("Failed to load trajectory data")
        return
    
    # Load selected pairs from CSV
    logger.info("Loading selected car-following pairs from CSV...")
    selected_pairs = load_selected_pairs(pairs_file)
    
    if len(selected_pairs) == 0:
        logger.error("Failed to load selected pairs")
        return
    
    # Process and visualize pairs
    logger.info("Processing and visualizing selected pairs...")
    pair_data = process_pair_data(data, selected_pairs)
    
    if len(pair_data) == 0:
        logger.error("No valid pair data generated for visualization")
        return
    
    plot_pair_visualizations(pair_data, output_dir)
    logger.info(f"Visualizations completed")
    
    # Calibrate CTH-RV model for each pair
    logger.info("Calibrating CTH-RV model for each pair...")
    calibration_results = calibrate_cth_rv_model(pair_data, output_dir)
    
    # Print calibration summary
    print("\nCalibration Results Summary:")
    print("=" * 100)
    print(f"{'Pair ID':<15} {'Alpha (α)':<10} {'Tau (τ)':<10} {'Beta (β)':<10} " +
          f"{'Spacing RMSE':<15} {'Speed RMSE':<15} {'String Stable':<12}")
    print("-" * 100)
    
    for pair_id, results in calibration_results.items():
        print(f"{pair_id:<15} {results['alpha']:<10.4f} {results['tau']:<10.4f} {results['beta']:<10.4f} " +
              f"{results['spacing_rmse']:<15.2f} {results['speed_rmse']:<15.2f} {results['stability_status']}")
    
    print("=" * 100)
    logger.info("Analysis complete!")

if __name__ == "__main__":
    main()

2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-02-23 19:27:58,759 - __main__ - INFO - Loading trajectory data...
2025-0


Calibration Results Summary:
Pair ID         Alpha (α)  Tau (τ)    Beta (β)   Spacing RMSE    Speed RMSE      String Stable
----------------------------------------------------------------------------------------------------
120.0-125.0     0.0001     1.5000     0.3975     0.14            0.73            UNSTABLE
1087.0-1101.0   -0.0063    2.4557     0.1304     0.34            0.59            STABLE
2066.0-2074.0   0.0103     1.5000     0.1792     0.25            1.42            UNSTABLE
260.0-267.0     0.0229     1.5000     -0.0039    0.15            0.38            UNSTABLE
1463.0-1478.0   0.9000     2.5000     -0.6750    0.18            1.52            STABLE


In [3]:
plt.style.available

['Solarize_Light2',
 '_classic_test_patch',
 '_mpl-gallery',
 '_mpl-gallery-nogrid',
 'bmh',
 'classic',
 'dark_background',
 'fast',
 'fivethirtyeight',
 'ggplot',
 'grayscale',
 'petroff10',
 'seaborn-v0_8',
 'seaborn-v0_8-bright',
 'seaborn-v0_8-colorblind',
 'seaborn-v0_8-dark',
 'seaborn-v0_8-dark-palette',
 'seaborn-v0_8-darkgrid',
 'seaborn-v0_8-deep',
 'seaborn-v0_8-muted',
 'seaborn-v0_8-notebook',
 'seaborn-v0_8-paper',
 'seaborn-v0_8-pastel',
 'seaborn-v0_8-poster',
 'seaborn-v0_8-talk',
 'seaborn-v0_8-ticks',
 'seaborn-v0_8-white',
 'seaborn-v0_8-whitegrid',
 'tableau-colorblind10']

**Lane Change Analysis :** Count the lane-change occurrences in the dataset. Analyze where and when these lane changes occur, identifying any observable trends.

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple

class LaneChangeAnalyzer:
    """
    Analyzes lane changes in vehicle trajectory data to identify patterns and trends.
    """
    def __init__(self, min_duration: float = 0.5):
        """
        Initialize analyzer with minimum duration threshold for valid lane changes.
        
        Args:
            min_duration: Minimum duration in seconds for a valid lane change
        """
        self.min_duration = min_duration
        self.min_frames = int(min_duration * 10)  # Convert to frames (0.1s intervals)
        self.lane_changes = []
        
    def identify_lane_changes(self, df: pd.DataFrame) -> List[Dict]:
        """
        Identifies lane change events in trajectory data.
        
        Args:
            df: DataFrame with columns for Vehicle_ID, Frame_ID, Lane_ID
            
        Returns:
            List of dictionaries containing lane change information
        """
        lane_changes = []
        
        # Process each vehicle's trajectory
        for vehicle_id in df['Vehicle_ID'].unique():
            vehicle_data = df[df['Vehicle_ID'] == vehicle_id].sort_values('Frame_ID')
            
            # Track lane changes
            prev_lane = None
            start_frame = None
            from_lane = None
            
            for idx, row in vehicle_data.iterrows():
                curr_lane = row['Lane_ID']
                
                if prev_lane is not None and curr_lane != prev_lane:
                    if start_frame is None:
                        # Start of lane change
                        start_frame = row['Frame_ID']
                        from_lane = prev_lane
                    elif curr_lane != from_lane:
                        # End of lane change
                        duration = (row['Frame_ID'] - start_frame) * 0.1  # Convert to seconds
                        
                        if duration >= self.min_duration:
                            lane_changes.append({
                                'vehicle_id': vehicle_id,
                                'start_frame': start_frame,
                                'end_frame': row['Frame_ID'],
                                'from_lane': from_lane,
                                'to_lane': curr_lane,
                                'duration': duration,
                                'position': row['LocalY'],
                                'speed': row['Mean Speed']
                            })
                        
                        # Reset for next lane change
                        start_frame = None
                        from_lane = None
                
                prev_lane = curr_lane
        
        self.lane_changes = lane_changes
        return lane_changes
    
    def analyze_patterns(self) -> Dict:
        """
        Analyzes patterns in identified lane changes.
        
        Returns:
            Dictionary containing various statistics and patterns
        """
        if not self.lane_changes:
            return {}
            
        df_changes = pd.DataFrame(self.lane_changes)
        
        # Calculate statistics
        stats = {
            'total_changes': len(self.lane_changes),
            'avg_duration': df_changes['duration'].mean(),
            'avg_speed': df_changes['speed'].mean(),
            'by_lane': self._analyze_lane_distribution(df_changes),
            'by_position': self._analyze_spatial_distribution(df_changes),
            'by_time': self._analyze_temporal_distribution(df_changes)
        }
        
        return stats
    
    def _analyze_lane_distribution(self, df: pd.DataFrame) -> Dict:
        """Analyzes distribution of lane changes across lanes."""
        transitions = pd.crosstab(df['from_lane'], df['to_lane'])
        
        # Calculate common lane change patterns
        patterns = {
            'left_changes': df[df['to_lane'] < df['from_lane']].shape[0],
            'right_changes': df[df['to_lane'] > df['from_lane']].shape[0],
            'most_common_from': df['from_lane'].mode().iloc[0],
            'most_common_to': df['to_lane'].mode().iloc[0]
        }
        
        return {'transitions': transitions.to_dict(), 'patterns': patterns}
    
    def _analyze_spatial_distribution(self, df: pd.DataFrame) -> Dict:
        """Analyzes where lane changes occur along the road section."""
        # Create position bins (every 50 meters)
        bins = np.arange(0, df['position'].max() + 50, 50)
        position_dist = pd.cut(df['position'], bins=bins).value_counts().sort_index()
        
        return {
            'position_counts': position_dist.to_dict(),
            'hotspots': self._identify_hotspots(position_dist)
        }
    
    def _analyze_temporal_distribution(self, df: pd.DataFrame) -> Dict:
        """Analyzes when lane changes occur."""
        # Analyze by time window (every 30 seconds)
        time_bins = np.arange(0, df['start_frame'].max() * 0.1 + 30, 30)
        time_dist = pd.cut(df['start_frame'] * 0.1, bins=time_bins).value_counts().sort_index()
        
        return {
            'time_counts': time_dist.to_dict(),
            'peak_period': time_bins[time_dist.argmax()]
        }
    
    def _identify_hotspots(self, position_dist: pd.Series) -> List[Tuple[float, float]]:
        """Identifies spatial hotspots of lane changing activity."""
        mean = position_dist.mean()
        std = position_dist.std()
        hotspots = position_dist[position_dist > (mean + std)].index.tolist()
        return [(interval.left, interval.right) for interval in hotspots]
    
    def plot_analysis(self) -> None:
        """Creates visualizations of lane change patterns."""
        if not self.lane_changes:
            return
            
        df_changes = pd.DataFrame(self.lane_changes)
        
        # Create figure with subplots
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. Lane transition heatmap
        transitions = pd.crosstab(df_changes['from_lane'], df_changes['to_lane'])
        sns.heatmap(transitions, annot=True, fmt='d', cmap='YlOrRd', ax=ax1)
        ax1.set_title('Lane Change Transitions')
        ax1.set_xlabel('To Lane')
        ax1.set_ylabel('From Lane')
        
        # 2. Spatial distribution
        position_bins = np.arange(0, df_changes['position'].max() + 50, 50)
        df_changes['position_bin'] = pd.cut(df_changes['position'], bins=position_bins)
        position_counts = df_changes['position_bin'].value_counts().sort_index()
        position_counts.plot(kind='bar', ax=ax2)
        ax2.set_title('Lane Changes by Location')
        ax2.set_xlabel('Position along road (m)')
        ax2.set_ylabel('Number of lane changes')
        
        # 3. Temporal distribution
        time_bins = np.arange(0, df_changes['start_frame'].max() * 0.1 + 30, 30)
        df_changes['time_bin'] = pd.cut(df_changes['start_frame'] * 0.1, bins=time_bins)
        time_counts = df_changes['time_bin'].value_counts().sort_index()
        time_counts.plot(kind='bar', ax=ax3)
        ax3.set_title('Lane Changes over Time')
        ax3.set_xlabel('Time (s)')
        ax3.set_ylabel('Number of lane changes')
        
        # 4. Speed distribution
        sns.boxplot(x='from_lane', y='speed', data=df_changes, ax=ax4)
        ax4.set_title('Speed Distribution during Lane Changes')
        ax4.set_xlabel('From Lane')
        ax4.set_ylabel('Speed (m/s)')
        
        plt.tight_layout()
        plt.savefig('lane_change_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()

def analyze_ngsim_lane_changes(file_path: str) -> None:
    """
    Analyzes lane changes in NGSIM data and prints summary statistics.
    
    Args:
        file_path: Path to the NGSIM trajectory data file
    """
    # Read data
    df = pd.read_csv("D:/ASU Academics/Traffic Flow Theroy/MP-1/Reconstructed NGSIM I80-1 data/Data/DATA (NO MOTORCYCLES).txt", delimiter=r'\s+', header=None,
                     names=['Vehicle_ID', 'Frame_ID', 'Lane_ID', 'LocalY',
                           'Mean Speed', 'Mean Acceleration', 'Vehicle_Length',
                           'Vehicle_Class', 'Follower_ID', 'Leader_ID'])
    
    # Initialize analyzer
    analyzer = LaneChangeAnalyzer(min_duration=0.5)
    
    # Identify and analyze lane changes
    lane_changes = analyzer.identify_lane_changes(df)
    patterns = analyzer.analyze_patterns()
    
    # Print summary
    print("\nLane Change Analysis Summary:")
    print(f"Total lane changes: {patterns['total_changes']}")
    print(f"Average duration: {patterns['avg_duration']:.2f} seconds")
    print(f"Average speed during lane change: {patterns['avg_speed']:.2f} m/s")
    
    # Print lane change patterns
    lane_dist = patterns['by_lane']['patterns']
    print("\nLane change patterns:")
    print(f"Left lane changes: {lane_dist['left_changes']}")
    print(f"Right lane changes: {lane_dist['right_changes']}")
    print(f"Most common origin lane: {lane_dist['most_common_from']}")
    print(f"Most common destination lane: {lane_dist['most_common_to']}")
    
    # Create visualizations
    analyzer.plot_analysis()
    
    
import pandas as pd
import numpy as np
import logging
from datetime import datetime

# Configure logging to track the analysis progress
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def load_ngsim_data(file_path: str) -> pd.DataFrame:
    """
    Loads and preprocesses NGSIM trajectory data.
    
    Args:
        file_path: Path to the NGSIM data file
        
    Returns:
        DataFrame with processed trajectory data
    """
    try:
        # Read the space-delimited data file
        df = pd.read_csv(file_path, delimiter=r'\s+', header=None,
                        names=['Vehicle_ID', 'Frame_ID', 'Lane_ID', 'LocalY',
                              'Mean Speed', 'Mean Acceleration', 'Vehicle_Length',
                              'Vehicle_Class', 'Follower_ID', 'Leader_ID'])
        
        logger.info(f"Successfully loaded data with {len(df)} rows")
        return df
        
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        return pd.DataFrame()

def main():
    """
    Main function to run the lane change analysis on NGSIM data.
    """
    # Start timing the analysis
    start_time = datetime.now()
    logger.info("Starting lane change analysis...")
    
    try:
        # Load data
        file_path = "D:/ASU Academics/Traffic Flow Theroy/MP-1/Reconstructed NGSIM I80-1 data/Data/DATA (NO MOTORCYCLES).txt"
        df = load_ngsim_data(file_path)
        
        if df.empty:
            logger.error("Failed to load data. Exiting...")
            return
            
        # Initialize the analyzer with configured parameters
        analyzer = LaneChangeAnalyzer(
            min_duration=0.5  # Minimum 0.5 seconds for a lane change
        )
        
        # Identify lane changes
        logger.info("Identifying lane changes...")
        lane_changes = analyzer.identify_lane_changes(df)
        
        # Analyze patterns
        logger.info("Analyzing lane change patterns...")
        patterns = analyzer.analyze_patterns()
        
        # Print comprehensive analysis results
        print("\n" + "="*50)
        print("NGSIM Lane Change Analysis Results")
        print("="*50)
        
        print("\nOverall Statistics:")
        print(f"Total lane changes: {patterns['total_changes']}")
        print(f"Average duration: {patterns['avg_duration']:.2f} seconds")
        print(f"Average speed: {patterns['avg_speed']:.2f} m/s")
        
        # Print lane change patterns
        lane_dist = patterns['by_lane']['patterns']
        print("\nLane Change Patterns:")
        print(f"Left lane changes: {lane_dist['left_changes']}")
        print(f"Right lane changes: {lane_dist['right_changes']}")
        print(f"Most common origin lane: {lane_dist['most_common_from']}")
        print(f"Most common destination lane: {lane_dist['most_common_to']}")
        
        # Create visualizations
        logger.info("Generating visualizations...")
        analyzer.plot_analysis()
        
        # Calculate execution time
        execution_time = datetime.now() - start_time
        logger.info(f"Analysis completed in {execution_time.total_seconds():.2f} seconds")
        
        print("\nAnalysis complete! Visualizations saved as 'lane_change_analysis.png'")
        
    except Exception as e:
        logger.error(f"Error during analysis: {e}")
        raise

if __name__ == "__main__":
    main()

2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...
2025-02-23 23:18:48,027 - __main__ - INFO - Starting lane change analysis...


NGSIM Lane Change Analysis Results

Overall Statistics:
Total lane changes: 164
Average duration: 15.44 seconds
Average speed: 7.95 m/s

Lane Change Patterns:
Left lane changes: 140
Right lane changes: 24
Most common origin lane: 7.0
Most common destination lane: 5.0


2025-02-23 23:19:35,503 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2025-02-23 23:19:35,503 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2025-02-23 23:19:37,716 - __main__ - INFO - Analysis completed in 49.69 seconds
2025-02-23 23:19:37,716 - __main__ - INFO - Analysis completed in 49.69 seconds
2025-02-23 23:19:37,716 - __main__ - INFO - Analysis completed in 49.69 seconds
2025-02-23 23:19:37,716 - __main__ - INFO - Analysis completed in 49.69 seconds
2025-02-23 23:19:37,716 - __main__ - INFO - Analysis completed in 49.69 seconds
2025-02-23 23:19:37,716 - __main__ - INFO - Analysis completed in 49.69 seconds
2025-02-23 23:19:37,716 - __main__ - INFO - Analysis completed in 49.69 se


Analysis complete! Visualizations saved as 'lane_change_analysis.png'


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class LaneChangeAnalyzer:
    def __init__(self, min_duration: float = 0.5):
        self.min_duration = min_duration
        self.min_frames = int(min_duration * 10)
        self.lane_changes = []
        
    def analyze_trajectories(self, df: pd.DataFrame) -> None:
        """Analyzes trajectory data to identify lane changes"""
        # First, rename columns to match our expected format
        df.columns = ['Vehicle_ID', 'Frame_ID', 'Lane_ID', 'LocalY',
                     'Speed', 'Acceleration', 'Vehicle_Length',
                     'Vehicle_Class', 'Follower_ID', 'Leader_ID']
        
        # Group by vehicle
        for vehicle_id in df['Vehicle_ID'].unique():
            # Get vehicle's trajectory
            vehicle_data = df[df['Vehicle_ID'] == vehicle_id].sort_values('Frame_ID')
            
            # Initialize variables for tracking lane changes
            prev_lane = None
            start_frame = None
            from_lane = None
            
            # Analyze frame by frame
            for idx, row in vehicle_data.iterrows():
                curr_lane = row['Lane_ID']
                
                if prev_lane is not None and curr_lane != prev_lane:
                    if start_frame is None:
                        # Start of lane change
                        start_frame = row['Frame_ID']
                        from_lane = prev_lane
                    elif curr_lane != from_lane:
                        # End of lane change
                        end_frame = row['Frame_ID']
                        duration = (end_frame - start_frame) * 0.1
                        
                        if duration >= self.min_duration:
                            self.lane_changes.append({
                                'vehicle_id': vehicle_id,
                                'start_frame': start_frame,
                                'end_frame': end_frame,
                                'from_lane': from_lane,
                                'to_lane': curr_lane,
                                'position': row['LocalY'],
                                'speed': row['Speed'],
                                'duration': duration,
                                'direction': 'left' if curr_lane > from_lane else 'right'
                            })
                        
                        # Reset for next lane change but immediately start tracking a potential new lane change
                        start_frame = row['Frame_ID']  # Start tracking a new lane change immediately
                        from_lane = prev_lane  # The lane we just left might be the start of a new lane change
                
                prev_lane = curr_lane
    
    def generate_statistics(self) -> dict:
        """Generates summary statistics of lane changes"""
        if not self.lane_changes:
            return {
                'total_changes': 0,
                'direction_counts': {'left': 0, 'right': 0},
                'avg_duration': 0,
                'speed_stats': {'mean': 0, 'std': 0, 'min': 0, 'max': 0}
            }
            
        df_changes = pd.DataFrame(self.lane_changes)
        
        stats = {
            'total_changes': len(self.lane_changes),
            'direction_counts': df_changes['direction'].value_counts().to_dict(),
            'avg_duration': df_changes['duration'].mean(),
            'speed_stats': {
                'mean': df_changes['speed'].mean(),
                'std': df_changes['speed'].std(),
                'min': df_changes['speed'].min(),
                'max': df_changes['speed'].max()
            },
            'lane_transitions': pd.crosstab(
                df_changes['from_lane'],
                df_changes['to_lane']
            ).to_dict(),
            'position_dist': pd.cut(
                df_changes['position'],
                bins=np.arange(0, df_changes['position'].max() + 100, 100)
            ).value_counts().sort_index().to_dict()
        }
        
        return stats
    
    def plot_analysis(self, stats: dict) -> None:
        """Creates visualizations of lane change patterns"""
        fig = plt.figure(figsize=(15, 10))
        
        # 1. Direction Distribution (Top Left)
        plt.subplot(221)
        directions = list(stats['direction_counts'].keys())
        counts = list(stats['direction_counts'].values())
        plt.bar(directions, counts)
        plt.title('Lane Change Direction Distribution')
        plt.ylabel('Number of Lane Changes')
        
        # 2. Position Distribution (Top Right)
        plt.subplot(222)
        positions = list(stats['position_dist'].keys())
        position_counts = list(stats['position_dist'].values())
        plt.bar(range(len(positions)), position_counts)
        plt.title('Lane Change Position Distribution')
        plt.xlabel('Position (100m segments)')
        plt.ylabel('Number of Lane Changes')
        
        # 3. Lane Transitions Heatmap (Bottom Left)
        plt.subplot(223)
        if self.lane_changes:
            df_changes = pd.DataFrame(self.lane_changes)
            transition_matrix = pd.crosstab(
                df_changes['from_lane'],
                df_changes['to_lane']
            )
            sns.heatmap(transition_matrix, annot=True, fmt='d', cmap='YlOrRd')
            plt.title('Lane Change Transitions')
            plt.xlabel('To Lane')
            plt.ylabel('From Lane')
                    
        # 4. Speed Distribution (Bottom Right)
        plt.subplot(224)
        if self.lane_changes:
            speeds = [lc['speed'] for lc in self.lane_changes]
            plt.hist(speeds, bins=20)
            plt.title('Speed During Lane Changes')
            plt.xlabel('Speed (m/s)')
            plt.ylabel('Frequency')
                    
        plt.tight_layout()
        plt.savefig('lane_change_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()

def main():
    # Read data file
    print("Reading data file...")
    df = pd.read_csv("D:/ASU Academics/Traffic Flow Theroy/MP-1/Reconstructed NGSIM I80-1 data/Data/DATA (NO MOTORCYCLES).txt", delimiter='\s+', header=None)
    
    # Initialize analyzer
    print("Analyzing lane changes...")
    analyzer = LaneChangeAnalyzer(min_duration=0.5)
    
    # Analyze trajectories
    analyzer.analyze_trajectories(df)
    
    # Generate statistics
    print("Generating statistics...")
    stats = analyzer.generate_statistics()
    
    # Print summary
    print("\nLane Change Analysis Summary:")
    print(f"Total lane changes: {stats['total_changes']}")
    print("\nDirection distribution:")
    for direction, count in stats['direction_counts'].items():
        print(f"{direction}: {count} ({count/stats['total_changes']*100:.1f}%)")
    print(f"\nAverage duration: {stats['avg_duration']:.2f} seconds")
    print("\nSpeed statistics during lane changes:")
    for stat, value in stats['speed_stats'].items():
        print(f"{stat}: {value:.2f} m/s")
    
    # Create visualizations
    print("\nCreating visualizations...")
    analyzer.plot_analysis(stats)
    print("Analysis complete. Visualizations saved as 'lane_change_analysis.png'")

if __name__ == "__main__":
    main()

Reading data file...
Analyzing lane changes...
Generating statistics...

Lane Change Analysis Summary:
Total lane changes: 204

Direction distribution:
right: 172 (84.3%)
left: 32 (15.7%)

Average duration: 15.26 seconds

Speed statistics during lane changes:
mean: 7.86 m/s
std: 2.44 m/s
min: 0.97 m/s
max: 14.77 m/s

Creating visualizations...
Analysis complete. Visualizations saved as 'lane_change_analysis.png'


In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from datetime import datetime
from typing import Dict, List, Tuple, Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class TrafficLaneChangeAnalyzer:
    """
    Analyzes lane changing behavior in traffic trajectory data with a focus on 
    traffic flow characteristics and patterns.
    
    This class implements traffic engineering principles to analyze:
    - Mandatory vs discretionary lane changes
    - Relationship with traffic density and flow conditions
    - Spatial and temporal patterns of lane changing behavior
    """
    
    def __init__(self, merge_zone_start: float = 200, merge_zone_end: float = 250):
        """
        Initialize analyzer with study section characteristics.
        
        Args:
            merge_zone_start: Start position of merge area (meters)
            merge_zone_end: End position of merge area (meters)
        """
        self.merge_zone = (merge_zone_start, merge_zone_end)
        self.lane_changes = []
        
        # Traffic analysis parameters
        self.segment_length = 50  # Analysis segment length (meters)
        self.time_window = 30     # Analysis time window (seconds)
        
        # Density thresholds based on HCM guidelines (vehicles/km)
        self.density_levels = {
            'Free flow': (0, 12),
            'Stable': (12, 20),
            'Near capacity': (20, 28),
            'Unstable': (28, 35),
            'Congested': (35, float('inf'))
        }
    
    def generate_traffic_report(self, patterns: Dict) -> str:
        """
        Generates a comprehensive traffic engineering report of the analysis results.
        
        Args:
            patterns: Dictionary containing analysis patterns and statistics
            
        Returns:
            str: Formatted report text
        """
        # Create section separator
        separator = "\n" + "="*80 + "\n"
        
        # Header
        report = "NGSIM Traffic Lane Change Analysis Report"
        report += "\nDate: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        report += separator
        
        # Executive Summary
        report += "Executive Summary\n" + "-"*20 + "\n"
        stats = patterns['overall_stats']
        report += f"Total analyzed lane changes: {stats['total_changes']}\n"
        report += f"Average speed during lane changes: {stats['avg_speed']:.2f} m/s\n"
        report += f"Average traffic density: {stats['avg_density']:.2f} veh/km\n\n"
        
        # Lane Change Classification
        report += "Lane Change Classification\n" + "-"*20 + "\n"
        for type_, count in stats['by_type'].items():
            percentage = (count/stats['total_changes'])*100
            report += f"{type_} lane changes: {count} ({percentage:.1f}%)\n"
        
        # Traffic Flow Characteristics
        report += "\nTraffic Flow Characteristics\n" + "-"*20 + "\n"
        traffic = patterns['traffic_conditions']
        
        report += "Lane changes by traffic density level:\n"
        for level, analysis in traffic['density_analysis'].items():
            report += f"\n{level}:\n"
            report += f"  Number of lane changes: {analysis['count']}\n"
            report += f"  Percentage: {analysis['percentage']:.1f}%\n"
            report += f"  Average speed: {analysis['avg_speed']:.2f} m/s\n"
        
        # Correlation Analysis
        corr = traffic['correlation']
        report += "\nCorrelation Analysis:\n"
        report += f"Speed-Density correlation: {corr['density_speed']:.3f}\n"
        report += f"Proportion of discretionary changes: {corr['density_changes']:.3f}\n"
        
        # Spatial Analysis
        report += "\nSpatial Analysis\n" + "-"*20 + "\n"
        spatial = patterns['spatial_temporal']
        
        report += "Peak lane change locations:\n"
        for peak in spatial['peak_locations']:
            report += f"  {peak['location']}: {peak['count']} lane changes\n"
        
        report += "\nMerge Area Analysis:\n"
        merge_changes = len([lc for lc in self.lane_changes if 
                          self.merge_zone[0] <= lc['position'] <= self.merge_zone[1]])
        merge_percentage = (merge_changes/stats['total_changes'])*100
        report += f"Lane changes in merge area: {merge_changes} ({merge_percentage:.1f}%)\n"
        
        # Analysis Notes
        report += "\nAnalysis Notes\n" + "-"*20 + "\n"
        report += "1. Lane changes are classified as mandatory, discretionary, or merging.\n"
        report += "2. Traffic density calculated using Edie's generalized definition.\n"
        report += "3. Merge zone defined as 200-250m from section start.\n"
        report += "4. Analysis uses 30-second time windows for stability.\n"
        
        return report
    
    def analyze_trajectories(self, df: pd.DataFrame) -> None:
        """
        Analyzes vehicle trajectory data to identify and classify lane changes.
        
        This method:
        1. Calculates traffic conditions (density, flow)
        2. Identifies lane change events
        3. Classifies lane changes based on location and traffic conditions
        
        Args:
            df: DataFrame with trajectory data
        """
        # Rename columns for consistency
        df.columns = ['Vehicle_ID', 'Frame_ID', 'Lane_ID', 'LocalY',
                     'Speed', 'Acceleration', 'Vehicle_Length',
                     'Vehicle_Class', 'Follower_ID', 'Leader_ID']
        
        # Calculate traffic metrics
        df['Time'] = df['Frame_ID'] * 0.1  # Convert frames to seconds
        df['Density'] = self._calculate_local_density(df)
        
        # Process each vehicle's trajectory
        for vehicle_id in df['Vehicle_ID'].unique():
            vehicle_data = df[df['Vehicle_ID'] == vehicle_id].sort_values('Frame_ID')
            self._analyze_vehicle_lane_changes(vehicle_data)
    
    def _calculate_local_density(self, df: pd.DataFrame) -> pd.Series:
        """
        Calculates local traffic density using Edie's generalized definition.
        
        The method:
        1. Divides road into segments
        2. Uses time windows for stable measurements
        3. Handles multiple vehicles in same frame/lane properly
        
        Args:
            df: DataFrame with trajectory data
                
        Returns:
            Series with density values for each vehicle/frame combination
        """
        # Create spatial segments
        segment_bounds = np.arange(
            df['LocalY'].min(),
            df['LocalY'].max() + self.segment_length,
            self.segment_length
        )
        df['Segment'] = pd.cut(df['LocalY'], bins=segment_bounds, labels=False)
        
        # Create time windows
        time_bounds = np.arange(
            df['Time'].min(),
            df['Time'].max() + self.time_window,
            self.time_window
        )
        df['TimeWindow'] = pd.cut(df['Time'], bins=time_bounds, labels=False)
        
        # Calculate vehicle counts and densities for each space-time region
        density_calc = (
            df.groupby(['TimeWindow', 'Segment', 'Lane_ID'])
            .agg({
                'Vehicle_ID': 'nunique',  # Count unique vehicles
                'Frame_ID': lambda x: list(x)  # Keep all frame IDs
            })
            .reset_index()
        )
        
        # Calculate density (vehicles per km)
        density_calc['Density'] = density_calc['Vehicle_ID'] / (self.segment_length / 1000)
        
        # Create a mapping from (Frame_ID, Lane_ID) to density
        density_map = {}
        for _, row in density_calc.iterrows():
            for frame in row['Frame_ID']:
                density_map[(frame, row['Lane_ID'])] = row['Density']
        
        # Apply density values back to original DataFrame
        return df.apply(lambda x: density_map.get((x['Frame_ID'], x['Lane_ID']), 0), axis=1)
    
    def _identify_peak_locations(self, df: pd.DataFrame) -> List[Dict]:
        """
        Identifies locations with significant lane changing activity.
        
        The method:
        1. Creates position bins
        2. Counts lane changes in each bin
        3. Identifies peaks using statistical thresholds
        
        Args:
            df: DataFrame with lane change data
            
        Returns:
            List of dictionaries containing peak location information
        """
        # Create position bins (50m segments)
        bins = np.arange(0, df['position'].max() + 50, 50)
        labels = [f"{bins[i]:.0f}-{bins[i+1]:.0f}m" for i in range(len(bins)-1)]
        
        # Count lane changes in each bin
        position_counts = pd.cut(df['position'], bins=bins, labels=labels).value_counts()
        
        # Calculate threshold for peak identification
        mean_count = position_counts.mean()
        std_count = position_counts.std()
        threshold = mean_count + std_count
        
        # Identify peaks
        peaks = []
        for loc, count in position_counts[position_counts > threshold].items():
            peaks.append({
                'location': loc,
                'count': count,
                'percentage': (count/len(df))*100
            })
        
        return sorted(peaks, key=lambda x: x['count'], reverse=True)
    
    def _analyze_vehicle_lane_changes(self, vehicle_data: pd.DataFrame) -> None:
        """
        Identifies and classifies lane changes for a single vehicle.
        
        Args:
            vehicle_data: DataFrame with single vehicle's trajectory
        """
        prev_lane = None
        start_frame = None
        
        for idx, row in vehicle_data.iterrows():
            curr_lane = row['Lane_ID']
            
            if prev_lane is not None and curr_lane != prev_lane:
                if start_frame is None:
                    # Start of lane change
                    start_frame = row['Frame_ID']
                    start_pos = row['LocalY']
                    start_time = row['Time']
                    density_at_start = row['Density']
                else:
                    # Classify and record the lane change
                    self._record_lane_change(
                        vehicle_id=row['Vehicle_ID'],
                        start_frame=start_frame,
                        end_frame=row['Frame_ID'],
                        from_lane=prev_lane,
                        to_lane=curr_lane,
                        position=row['LocalY'],
                        start_pos=start_pos,
                        speed=row['Speed'],
                        density=density_at_start,
                        start_time=start_time
                    )
                    start_frame = None
            
            prev_lane = curr_lane
    
    def _record_lane_change(self, **kwargs) -> None:
        """Records a lane change with traffic-specific classification"""
        is_in_merge_zone = (self.merge_zone[0] <= kwargs['position'] <= self.merge_zone[1])
        is_merging = kwargs['from_lane'] == 7 or kwargs['to_lane'] == 7
        
        change_type = self._classify_lane_change(
            is_in_merge_zone=is_in_merge_zone,
            is_merging=is_merging,
            from_lane=kwargs['from_lane'],
            to_lane=kwargs['to_lane']
        )
        
        self.lane_changes.append({
            'vehicle_id': kwargs['vehicle_id'],
            'start_frame': kwargs['start_frame'],
            'end_frame': kwargs['end_frame'],
            'from_lane': kwargs['from_lane'],
            'to_lane': kwargs['to_lane'],
            'position': kwargs['position'],
            'start_position': kwargs['start_pos'],
            'speed': kwargs['speed'],
            'time': kwargs['start_time'],
            'density': kwargs['density'],
            'type': change_type,
            'direction': 'left' if kwargs['to_lane'] > kwargs['from_lane'] else 'right'
        })
    
    def _classify_lane_change(self, **kwargs) -> str:
        """
        Classifies lane changes based on traffic engineering criteria.
        
        Returns:
            'merge': Lane changes involving the merge lane
            'mandatory': Lane changes in merge zone
            'discretionary': Other lane changes
        """
        if kwargs['is_merging']:
            return 'merge'
        elif kwargs['is_in_merge_zone']:
            return 'mandatory'
        else:
            return 'discretionary'
    
    def analyze_patterns(self) -> Dict:
        """
        Generates comprehensive traffic analysis of lane changes.
        
        Returns:
            Dictionary containing analysis results
        """
        df_changes = pd.DataFrame(self.lane_changes)
        
        return {
            'overall_stats': self._calculate_overall_stats(df_changes),
            'change_types': self._analyze_change_types(df_changes),
            'spatial_temporal': self._analyze_spatial_temporal(df_changes),
            'traffic_conditions': self._analyze_traffic_conditions(df_changes)
        }
    
    def _calculate_overall_stats(self, df: pd.DataFrame) -> Dict:
        """Calculates overall lane change statistics"""
        return {
            'total_changes': len(df),
            'by_type': df['type'].value_counts().to_dict(),
            'by_direction': df['direction'].value_counts().to_dict(),
            'avg_speed': df['speed'].mean(),
            'avg_density': df['density'].mean()
        }
    
    def _analyze_change_types(self, df: pd.DataFrame) -> Dict:
        """Analyzes different types of lane changes"""
        return {
            'type_speed': df.groupby('type')['speed'].mean().to_dict(),
            'type_density': df.groupby('type')['density'].mean().to_dict(),
            'type_distribution': df.groupby(['type', 'direction']).size().to_dict()
        }
    
    def _analyze_spatial_temporal(self, df: pd.DataFrame) -> Dict:
        """Analyzes spatial and temporal patterns"""
        df['time_bin'] = pd.cut(df['time'], 
                              bins=np.arange(0, df['time'].max() + self.time_window, 
                                           self.time_window))
        
        return {
            'temporal': df.groupby('time_bin').size().to_dict(),
            'spatial': df.groupby(pd.cut(df['position'], bins=10)).size().to_dict(),
            'peak_locations': self._identify_peak_locations(df)
        }
    
    def _analyze_traffic_conditions(self, df: pd.DataFrame) -> Dict:
        """Analyzes relationship between traffic conditions and lane changes"""
        density_bins = [v[0] for v in self.density_levels.values()] + [float('inf')]
        df['DensityLevel'] = pd.cut(df['density'], 
                                   bins=density_bins,
                                   labels=self.density_levels.keys())
        
        changes_by_density = df.groupby('DensityLevel').size()
        total_changes = len(df)
        
        density_analysis = {
            level: {
                'count': count,
                'percentage': (count/total_changes) * 100,
                'avg_speed': df[df['DensityLevel'] == level]['speed'].mean()
            }
            for level, count in changes_by_density.items()
        }
        
        return {
            'density_analysis': density_analysis,
            'correlation': {
                'density_speed': df['density'].corr(df['speed']),
                'density_changes': len(df[df['type'] == 'discretionary']) / total_changes
            }
        }
    
    def plot_analysis(self) -> None:
        """Creates comprehensive traffic-focused visualizations"""
        if not self.lane_changes:
            return
            
        df_changes = pd.DataFrame(self.lane_changes)
        
        fig = plt.figure(figsize=(20, 15))
        
        # 1. Lane Change Types Distribution
        plt.subplot(331)
        sns.countplot(data=df_changes, x='type', hue='direction')
        plt.title('Lane Change Types and Directions')
        plt.xticks(rotation=45)
        
        # 2. Spatial Distribution
        plt.subplot(332)
        sns.histplot(data=df_changes, x='position', bins=20)
        plt.axvspan(self.merge_zone[0], self.merge_zone[1], 
                   color='red', alpha=0.2, label='Merge Zone')
        plt.title('Spatial Distribution of Lane Changes')
        plt.xlabel('Position (m)')
        plt.legend()
        
        # 3. Temporal Distribution
        plt.subplot(333)
        sns.histplot(data=df_changes, x='time', bins=20)
        plt.title('Temporal Distribution of Lane Changes')
        plt.xlabel('Time (s)')
        
        # 4. Speed vs Density
        plt.subplot(334)
        sns.scatterplot(data=df_changes, x='density', y='speed', hue='type')
        plt.title('Traffic Density vs Speed During Lane Changes')
        
        # 5. Lane Transitions
        plt.subplot(335)
        transitions = pd.crosstab(df_changes['from_lane'], df_changes['to_lane'])
        sns.heatmap(transitions, annot=True, fmt='d', cmap='YlOrRd')
        plt.title('Lane Transition Matrix')
        
        plt.tight_layout()
        plt.savefig('traffic_lane_change_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()

    
def main():
    """Main function to run the NGSIM traffic analysis"""
    start_time = datetime.now()
    logger.info("Starting NGSIM traffic analysis...")
    
    try:
        # Load data
        file_path = "D:/ASU Academics/Traffic Flow Theroy/MP-1/Reconstructed NGSIM I80-1 data/Data/DATA (NO MOTORCYCLES).txt"
        df = pd.read_csv(file_path, delimiter=r'\s+', header=None)
        
        if df.empty:
            logger.error("Empty data file")
            return
            
        # Initialize analyzer
        analyzer = TrafficLaneChangeAnalyzer(
            merge_zone_start=200,
            merge_zone_end=250
        )
        
        # Analyze trajectories
        logger.info("Analyzing vehicle trajectories...")
        analyzer.analyze_trajectories(df)
        
        # Generate analysis
        logger.info("Generating traffic analysis patterns...")
        patterns = analyzer.analyze_patterns()
        
        # Print summary
        print("\n" + "="*50)
        print("NGSIM Lane Change Analysis Results")
        print("="*50)
        
        stats = patterns['overall_stats']
        print(f"\nTotal lane changes: {stats['total_changes']}")
        print("\nBy Type:")
        for type_, count in stats['by_type'].items():
            print(f"  {type_}: {count} ({count/stats['total_changes']*100:.1f}%)")
        
        print("\nBy Direction:")
        for direction, count in stats['by_direction'].items():
            print(f"  {direction}: {count} ({count/stats['total_changes']*100:.1f}%)")
        
        # Traffic conditions analysis
        traffic_conditions = patterns['traffic_conditions']
        print("\nTraffic Condition Analysis:")
        print("\nDensity Level Distribution:")
        for level, stats in traffic_conditions['density_analysis'].items():
            print(f"  {level}:")
            print(f"    Count: {stats['count']} ({stats['percentage']:.1f}%)")
            print(f"    Average Speed: {stats['avg_speed']:.2f} m/s")
        
        # Generate detailed report
        report_text = analyzer.generate_traffic_report(patterns)
        with open('traffic_analysis_report.txt', 'w') as f:
            f.write(report_text)
        
        # Create visualizations
        logger.info("Creating traffic analysis visualizations...")
        analyzer.plot_analysis()
        
        # Log completion
        execution_time = datetime.now() - start_time
        logger.info(f"Analysis completed in {execution_time.total_seconds():.2f} seconds")
        logger.info("Results saved to traffic_analysis_report.txt")
        logger.info("Visualizations saved as traffic_lane_change_analysis.png")
        
    except Exception as e:
        logger.error(f"Analysis failed: {str(e)}")
        raise

if __name__ == "__main__":
    main()

2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting NGSIM traffic analysis...
2025-02-23 23:51:40,835 - __main__ - INFO - Starting


NGSIM Lane Change Analysis Results

Total lane changes: 173

By Type:
  discretionary: 127 (73.4%)
  mandatory: 38 (22.0%)
  merge: 8 (4.6%)

By Direction:
  right: 148 (85.5%)
  left: 25 (14.5%)

Traffic Condition Analysis:

Density Level Distribution:
  Free flow:
    Count: 0 (0.0%)
    Average Speed: nan m/s
  Stable:
    Count: 1 (0.6%)
    Average Speed: 1.75 m/s
  Near capacity:
    Count: 0 (0.0%)
    Average Speed: nan m/s
  Unstable:
    Count: 0 (0.0%)
    Average Speed: nan m/s
  Congested:
    Count: 172 (99.4%)
    Average Speed: 7.92 m/s


2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INFO - Analysis completed in 59.09 seconds
2025-02-23 23:52:39,930 - __main__ - INF

**Time-Space Diagram :** Plot the lane-by-lane time-space diagram for all the NGSIM trajectory data. Based on the diagram, discuss traffic conditions and patterns of congestion.

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import seaborn as sns

class TimeSpaceDiagramAnalyzer:
    def __init__(self, data_file: str):
        """Initialize analyzer with data file path"""
        self.df = pd.read_csv(data_file, delimiter='\s+', header=None,
                             names=['Vehicle_ID', 'Frame_ID', 'Lane_ID', 'LocalY',
                                   'Speed', 'Acceleration', 'Vehicle_Length',
                                   'Vehicle_Class', 'Follower_ID', 'Leader_ID'])
        
        # Convert Frame_ID to time in seconds
        self.df['Time'] = self.df['Frame_ID'] * 0.1
        
    def plot_lane_diagram(self, lane_id: int, ax=None, cmap='RdYlGn'):
        """Plot time-space diagram for a specific lane"""
        lane_data = self.df[self.df['Lane_ID'] == lane_id]
        
        lines = []
        colors = []
        
        for vehicle_id in lane_data['Vehicle_ID'].unique():
            vehicle_traj = lane_data[lane_data['Vehicle_ID'] == vehicle_id]
            if len(vehicle_traj) > 1:
                points = np.column_stack((vehicle_traj['Time'], 
                                       vehicle_traj['LocalY']))
                lines.append(points)
                colors.append(np.mean(vehicle_traj['Speed']))
        
        if not lines:
            return None
            
        lc = LineCollection(lines, cmap=plt.get_cmap(cmap))
        lc.set_array(np.array(colors))
        
        if ax is None:
            ax = plt.gca()
            
        line = ax.add_collection(lc)
        
        # Set axis limits
        times = lane_data['Time']
        positions = lane_data['LocalY']
        ax.set_xlim(times.min(), times.max())
        ax.set_ylim(positions.min(), positions.max())
        
        return line

    def create_full_diagram(self):
        """Create time-space diagrams for all lanes"""
        lanes = sorted(self.df['Lane_ID'].unique())
        n_lanes = len(lanes)
        
        fig, axes = plt.subplots(n_lanes, 1, figsize=(15, 4*n_lanes), sharex=True)
        if n_lanes == 1:
            axes = [axes]
            
        fig.suptitle('', fontsize=16, y=0.92)
        
        for ax, lane_id in zip(axes, lanes):
            line = self.plot_lane_diagram(lane_id, ax=ax)
            if line is not None:
                plt.colorbar(line, ax=ax, label='Speed (m/s)')
            
            ax.set_ylabel('Position (m)')
            ax.set_title(f'Lane {lane_id}')
            ax.grid(True, linestyle='--', alpha=0.7)
            
        axes[-1].set_xlabel('Time (seconds)')
        
        plt.tight_layout()
        return fig, axes

    def plot_congestion_heatmap(self):
        """Create a heatmap showing average speeds by lane and time"""
        # Calculate time bins (5-minute intervals)
        time_bins = np.arange(0, self.df['Time'].max() + 300, 300)
        lanes = sorted(self.df['Lane_ID'].unique())
        
        # Create speed matrix
        speed_matrix = np.zeros((len(lanes), len(time_bins)-1))
        
        for i, lane_id in enumerate(lanes):
            lane_data = self.df[self.df['Lane_ID'] == lane_id]
            
            for j, (t_start, t_end) in enumerate(zip(time_bins[:-1], time_bins[1:])):
                mask = (lane_data['Time'] >= t_start) & (lane_data['Time'] < t_end)
                avg_speed = lane_data[mask]['Speed'].mean()
                speed_matrix[i, j] = avg_speed if not np.isnan(avg_speed) else 0
        
        # Create figure
        fig, ax = plt.subplots(figsize=(15, 5))
        
        # Create heatmap
        im = ax.imshow(speed_matrix, 
                      aspect='auto',
                      cmap='RdYlGn',
                      extent=[0, self.df['Time'].max()/60, len(lanes)-0.5, -0.5])
        
        # Add colorbar
        plt.colorbar(im, ax=ax, label='Average Speed (m/s)')
        
        # Configure axes
        ax.set_yticks(range(len(lanes)))
        ax.set_yticklabels([f'Lane {lane}' for lane in lanes])
        
        # Add time labels (in minutes)
        time_ticks = np.linspace(0, self.df['Time'].max()/60, 10)
        ax.set_xticks(time_ticks)
        ax.set_xticklabels([f'{t:.0f}' for t in time_ticks])
        
        plt.title('Traffic Speed Heatmap')
        plt.xlabel('Time (minutes)')
        plt.ylabel('Lane')
        
        return fig, ax

    def analyze_congestion(self):
        """Analyze congestion patterns"""
        congestion_threshold = 10  # m/s
        
        # Calculate average speeds in 5-minute windows
        self.df['time_window'] = pd.cut(self.df['Time'], 
                                      bins=np.arange(0, self.df['Time'].max() + 300, 300))
        
        speed_stats = self.df.groupby(['Lane_ID', 'time_window'])['Speed'].agg([
            'mean', 'std', 'count'
        ]).reset_index()
        
        # Identify congestion
        congestion = speed_stats[speed_stats['mean'] < congestion_threshold]
        
        # Calculate overall statistics
        lane_stats = self.df.groupby('Lane_ID')['Speed'].agg([
            'mean', 'std', 'min', 'max'
        ]).round(2)
        
        return {
            'congestion_periods': congestion,
            'lane_statistics': lane_stats,
            'congestion_threshold': congestion_threshold
        }

def main():
    # Initialize analyzer
    analyzer = TimeSpaceDiagramAnalyzer("D:/ASU Academics/Traffic Flow Theroy/MP-1/Reconstructed NGSIM I80-1 data/Data/DATA (NO MOTORCYCLES).txt")
    
    # Create time-space diagrams
    print("Creating time-space diagrams...")
    fig_ts, axes_ts = analyzer.create_full_diagram()
    fig_ts.savefig('time_space_diagram.png', dpi=100, bbox_inches='tight')
    plt.close(fig_ts)
    
    # Create congestion heatmap
    print("Creating congestion heatmap...")
    fig_heat, ax_heat = analyzer.plot_congestion_heatmap()
    fig_heat.savefig('congestion_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close(fig_heat)
    
    # Analyze congestion
    print("\nAnalyzing congestion patterns...")
    stats = analyzer.analyze_congestion()
    
    # Print summary
    print("\nTraffic Analysis Summary:")
    print(f"\nCongestion threshold: {stats['congestion_threshold']} m/s")
    
    print("\nLane Statistics:")
    print(stats['lane_statistics'])
    
    print("\nCongestion Periods:")
    congestion = stats['congestion_periods']
    if not congestion.empty:
        for _, period in congestion.iterrows():
            print(f"Lane {period['Lane_ID']}: "
                  f"Time window {period['time_window']}, "
                  f"Average speed: {period['mean']:.1f} m/s")

if __name__ == "__main__":
    main()

Creating time-space diagrams...
Creating congestion heatmap...

Analyzing congestion patterns...


C:\Users\Raswanth\AppData\Local\Temp\ipykernel_41024\3670967326.py:129: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  speed_stats = self.df.groupby(['Lane_ID', 'time_window'])['Speed'].agg([



Traffic Analysis Summary:

Congestion threshold: 10 m/s

Lane Statistics:
          mean   std   min    max
Lane_ID                          
1        16.63  3.71  0.58  31.63
2         7.11  2.64  0.00  17.74
3         7.06  2.47  0.00  16.52
4         6.37  2.78  0.00  16.74
5         7.02  2.99  0.00  19.24
6         6.90  3.14  0.00  20.08
7         6.34  4.18  0.00  20.74
999      10.76  3.60  0.80  17.77

Congestion Periods:
Lane 2: Time window (0.0, 300.0], Average speed: 7.3 m/s
Lane 2: Time window (300.0, 600.0], Average speed: 7.5 m/s
Lane 2: Time window (600.0, 900.0], Average speed: 6.3 m/s
Lane 2: Time window (900.0, 1200.0], Average speed: 9.1 m/s
Lane 3: Time window (0.0, 300.0], Average speed: 7.6 m/s
Lane 3: Time window (300.0, 600.0], Average speed: 7.1 m/s
Lane 3: Time window (600.0, 900.0], Average speed: 6.4 m/s
Lane 3: Time window (900.0, 1200.0], Average speed: 8.2 m/s
Lane 4: Time window (0.0, 300.0], Average speed: 7.5 m/s
Lane 4: Time window (300.0, 600.0], A